<a href="https://colab.research.google.com/github/Shauryasawant/Shauryasawant/blob/main/Hyundai_Glovis1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
from google.colab import files

url = "https://www.glovis.net/Eng/network/contentsid/241/index.do"

# Send a GET request to the URL
response = requests.get(url)

# Check if request was successful
if response.status_code == 200:
    # Parse the HTML content
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find all unordered lists with class 'nwList global'
    nw_lists = soup.find_all('ul', class_='nwList global')

    # Initialize a list to store all location data
    all_locations = []

    # Iterate through each <ul> with class 'nwList global'
    for nw_list in nw_lists:
        # Find preceding <p class="subTit">Branch</p> or similar tag
        sub_title = nw_list.find_previous_sibling('p', class_='subTit')
        if sub_title:
            sub_title_text = sub_title.text.strip()
        else:
            sub_title_text = ""

        # Initialize a dictionary to store location data for this section
        section_locations = {
            'sub_title': sub_title_text,
            'locations': []
        }

        # Find all list items within the unordered list
        items = nw_list.find_all('li')

        for item in items:
            # Extract name and address from each item
            name = item.find('p', class_='name').text.strip()
            address = item.find('p', class_='address').text.strip()

            # Append the extracted data as a dictionary to the locations list
            section_locations['locations'].append({
                'name': name,
                'address': address
            })

        # Append this section's data to all_locations
        all_locations.append(section_locations)

    # Write the scraped data to a CSV file
    filename = 'locations.csv'
    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['sub_title', 'name', 'address']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        # Write headers
        writer.writeheader()

        # Write data
        for entry in all_locations:
            for location in entry['locations']:
                writer.writerow({
                    'sub_title': entry['sub_title'],
                    'name': location['name'],
                    'address': location['address']
                })

    # Download the CSV file in Google Colab
    files.download(filename)

    print(f"Data has been scraped and saved. Downloaded file: {filename}")

else:
    print("Failed to retrieve the webpage.")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Data has been scraped and saved. Downloaded file: locations.csv
